In [3]:
import pandas as pd
import re
import os

import mysql.connector
db = mysql.connector.connect(user='dkbrz', password='password', database='folklore_v2')
cur = db.cursor()

In [36]:
from unidecode import unidecode

In [32]:
get_available_language_codes()

['sr', 'mn', 'mk', 'bg', 'uk', 'hy', 'ka', 'ru', 'el', 'l1']

In [38]:
unidecode('росписи_в_палехской_церкви_(1)')

'rospisi_v_palekhskoi_tserkvi_(1)'

In [12]:
PATH = "/home/dkbrz/github/folklore/folklore_app/static/gallery/"

In [13]:
from collections import defaultdict

In [20]:
counter = defaultdict(int)
for root, dirs, files in os.walk(PATH):
#     print(root, dirs, files)
    for f in files:
        tags = root.replace(PATH, "")
        tags = tags.split("/")
        for t in tags:
            counter[t] += 1

In [20]:
cur.executemany("INSERT INTO glr_tags (tag) VALUES (%s)", [(i,) for i in counter.keys()])
db.commit()

In [40]:
data = []
idx = 1
for root, dirs, files in os.walk(PATH):
    for f in files:
        if f[-3:].lower() != 'rtf':
            tags = root.replace(PATH, "")
            data.append((idx, tags, f))
            idx += 1
        else:
            pass
#             print(f)

In [41]:
data[:5]

[(1, 'Pskov_region/obryad/obereg', 'RG-Komary-16-Sebezhsk.JPG'),
 (2, 'Pskov_region/obryad/obereg', 'AAN-Sebezh-1.jpg'),
 (3, 'Pskov_region/obryad/obereg', 'RG-Komary-15-Sebezhsk.JPG'),
 (4, 'Pskov_region/obryad/obereg', 'NVV-Sebezh-1.jpg'),
 (5, 'Pskov_region/obryad/obereg', 'NVV-Sebezh-2.jpg')]

In [42]:
cur.executemany("INSERT INTO glr_images (id, folder_path, image_name) VALUES (%s, %s, %s)", data)
db.commit()

In [43]:
cur.execute("SELECT tag, id FROM glr_tags")
tag_map = dict(cur.fetchall())

In [44]:
cur.execute("SELECT folder_path, id FROM glr_images")
data = cur.fetchall()

In [45]:
image_tags = []
for path, idx in data:
    tags = path.split("/")
    for t in tags:
        image_tags.append((idx, tag_map[t]))

In [46]:
cur.executemany("INSERT INTO glr_image_tags (id_image, id_tag) VALUES (%s, %s)", image_tags)
db.commit()

## tags

In [3]:
df = pd.read_excel("/home/dkbrz/Downloads/latinica_kirillica.xlsx")

In [6]:
cur.executemany("UPDATE glr_tags SET rus = %s WHERE tag = %s", df[["kirillica", "latinica"]].values.tolist())
db.commit()

## renaming files

In [39]:
for root, dirs, files in os.walk(PATH):
    for f in files:
#             fname = os.path.join(root, f)
#             fname_ok = fname.replace(" ", "_")
#             os.rename(fname, fname_ok)
        fname = os.path.join(root, f)
        fname_ok = unidecode(fname)
        os.rename(fname, fname_ok)
#         print(f)

## copy with new name id

In [12]:
from shutil import copyfile
from tqdm import tqdm

In [ ]:
cur.execute("SELECT id, concat(folder_path, '/', image_name) FROM glr_images;")
imgs = cur.fetchall()
imgs[:5]

In [14]:
PATH = "../folklore_app/static/gallery/"
PATH_NEW = "../folklore_app/static/gallery2/"

for idx, img in tqdm(imgs):
    old_name = os.path.join(PATH, img)
    new_name = os.path.join(PATH_NEW, str(idx) + '.' + img.split(".")[-1])
    copyfile(old_name, new_name)

100%|██████████| 2795/2795 [00:43<00:00, 64.54it/s] 


In [7]:
cur.execute("SELECT id, image_name FROM glr_images;")
imgs = cur.fetchall()
imgs[:5]

[(1, 'RG-Komary-16-Sebezhsk.JPG'),
 (2, 'AAN-Sebezh-1.jpg'),
 (3, 'RG-Komary-15-Sebezhsk.JPG'),
 (4, 'NVV-Sebezh-1.jpg'),
 (5, 'NVV-Sebezh-2.jpg')]

In [9]:
new_names = []
for idx, img in imgs:
    new = str(idx) + '.' + img.split(".")[-1]
    new_names.append((new, idx))
new_names[:5]

[('1.JPG', 1), ('2.jpg', 2), ('3.JPG', 3), ('4.jpg', 4), ('5.jpg', 5)]

In [11]:
cur.executemany("UPDATE glr_images SET image_id_name = %s WHERE id = %s", new_names)
db.commit()